In [1]:
from scipy.signal import wiener
import cv2
import numpy as np
from numpy import fft
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def winner_filter(img):
    mean = 0
    sigma = 25
    h, w, c = img.shape
    gauss = np.random.normal(mean,sigma,(h, w, c))
    noise_img = img + gauss
    noise_img = np.clip(noise_img,a_min=0,a_max=255)
    result_img = wiener(noise_img,mysize=[20, 20, 1], noise=0)
    return result_img

In [3]:
def motion_process(height, weight, k=0.001):
    h_uv = np.zeros((height, weight), dtype='complex128')
    for u in range(height):
        for v in range(weight):
            q = np.power((u ** 2 + v ** 2), (5.0 / 6.0))
            h_uv[u][v] = np.exp(-(k * q))
    return h_uv

In [4]:
def blur_noise(src, h_uv):
    noise = np.random.normal(0, 1, size=src.shape)
    f_uv = np.fft.fft2(src)
    arr = np.multiply(f_uv, h_uv) + noise
    return arr, noise

In [5]:
def winner(src, h_uv, k=None):
    if k is None:
        p_uv = np.conj(h_uv) / (np.abs(h_uv) ** 2)
    else:
        p_uv = np.conj(h_uv) / (np.abs(h_uv) ** 2 + 0.00005*k)
    rst = np.multiply(src, p_uv)
    return np.abs(np.fft.ifft2(rst))

In [6]:
def main(img_path):
    src = np.array(Image.open(img_path).convert("L"))
    height, weight = src.shape
    h_uv = motion_process(height, weight, k=0.001)
    noise_src, noise = blur_noise(src, h_uv)
    mov_noi_src = np.abs(np.fft.ifft2(noise_src))
    k = np.power(np.abs(np.fft.fft2(noise)), 2) / np.power(np.abs(np.fft.fft2(src)), 2)

    src_processed_winner = winner(noise_src, h_uv, k)
    # cv2.imwrite('../worker2/result/result_{}.png'.format('origin'), src)
    # cv2.imwrite('../worker2/result/result_{}.png'.format('noise'), mov_noi_src)
    cv2.imwrite('../worker2/result/result_{}.png'.format('best'), src_processed_winner)

In [7]:
if __name__ == '__main__':
    main('../images/menna.jpg')